In [1]:
import pdfplumber
import csv

In [2]:
pdf6 = pdfplumber.open("mttq_bidv_enter.pdf")

In [3]:
import re, json
from collections import defaultdict

with open('mttq_bidv.csv', 'w', newline='') as csvfile:
    fieldnames = ['transaction_code', 'amount', 'transaction_detail',
                    # 'test'
                 ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    i = 0
    transaction_id = 0
    
    for page in pdf6.pages:
        # print(page.page_number)
        start = False
        info = defaultdict(str)
        transactions = []
        
        date = None

        tables = page.extract_tables()
        for raw_transactions in tables:
            i += 1
            if i == 1:
                raw_transactions = raw_transactions[1:]
            # print(raw_transactions)
            # break
            for record in raw_transactions:
                try:
                    # print(len(record), record)
                    transaction_code, timestamp, amount, transaction_detail = record
                    assert(int(transaction_code) == transaction_id + 1)
                    transaction_id = int(transaction_code)
                    amount = int(amount.replace('.', ''))
                    transactions.append({
                        'transaction_code': f'bidv_{transaction_code}',
                        'amount': amount,
                        'transaction_detail': f'{timestamp} {transaction_detail}'.replace('\r\n', ' ').replace('\r', ' ').replace('\n', ' ')
                    })
                except Exception as e:
                        # print(transactions[-1])
                    # elif 'Foreign\nWorker\nSalary' in transaction_detail:
                    #     print(len(record), record)
                    # else:
                    #     print(transaction_detail)
                    print(len(record), record)
                    pass
    
        # print(page, len(transactions))
        for row in transactions:
            writer.writerow(row)
        # print(transactions)
        # break